<a href="https://colab.research.google.com/github/Manikandan-professional/Miscellaneous/blob/main/G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>